# 1. SAR Applications
## Feature Tracking on Petermann Glacier using Sentinel-1 SAR Imagery

Synthetic Aperture Radar (SAR) has become an invaluable tool for monitoring glacier dynamics in the Arctic and Antarctic, providing all-weather, day-and-night observations of ice motion. This tutorial demonstrates glacier velocity estimation using feature tracking techniques applied to Sentinel-1 SAR imagery.

The example below focuses on Petermann Glacier, one of Greenland's largest outlet glaciers located in the northwest region. We'll track the movement of identifiable features between two Sentinel-1 SAR images acquired on February 26, 2020, and November 4, 2020—a time span of approximately 8.5 months.

While operational glacier velocity mapping typically relies on speckle tracking (exploiting the coherent backscatter patterns inherent in SAR imagery), this tutorial uses visible surface features for tracking, which provides a more intuitive understanding of the underlying principles. The fundamental approach remains the same: identifying corresponding patterns or features between image pairs and calculating their displacement over time.

Given the extended temporal separation between our image pair, any coherence between speckle patterns has been lost due to changes in surface properties, ice deformation, and environmental conditions. This makes visible feature tracking the preferred approach for this particular case, while also serving as an excellent pedagogical example of displacement tracking techniques in SAR glaciology.

In [ ]:
!pip install scikit-image numpy matplotlib

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread


First, we define a function which computes the cross correlation between the reference window (patch1) and a patch of similar size in the search window (patch2), following the formula in the lecture slides: 


$C C(i, j)=\frac{\sum_{k, l}\left[s(i+k, j+l)-\mu_s\right]\left[r(k, l)-\mu_r\right]}{\sqrt{\sum_{k, l}\left[s(i+k, j+l)-\mu_s\right]^2 \sum_{k, l}\left[r(k, l)-\mu_r\right]^2}}$

In [ ]:

def compute_normalized_cross_correlation(patch1, patch2):
    # Subtract the mean from each patch
    patch1_mean = np.mean(patch1)
    patch2_mean = np.mean(patch2)
    patch1_normalized = patch1 - patch1_mean
    patch2_normalized = patch2 - patch2_mean

    # Compute the cross-correlation
    cross_correlation = np.sum(patch1_normalized * patch2_normalized)

    # Compute the normalization factor
    normalization_factor = np.sqrt(np.sum(patch1_normalized ** 2) * np.sum(patch2_normalized ** 2))

    # Compute the normalized cross-correlation
    normalized_cross_correlation = cross_correlation / normalization_factor
    return normalized_cross_correlation

Now we load the images

In [ ]:
# Read SAR images from TIF files
image1 = imread('Data/SAR/S1A_IW_GRDH_1SDH_20200226_subset_TC.tif')
image2 = imread('Data/SAR/S1A_IW_GRDH_1SDH_20201104_subset_TC.tif')

Here we define the reference window (small) and the seach window (large). 

In [ ]:
# Define window sizes
n1 = 40  # small window size in image 1
n2 = 500  # large window size in image 2

# Define the coordinates for the center of the small window in image 1
x1_center = 2526
y1_center = 2234

# Extract the small window in image 1
x1_start = x1_center - n1 // 2
x1_end = x1_start + n1
y1_start = y1_center - n1 // 2
y1_end = y1_start + n1
patch1 = image1[x1_start:x1_end, y1_start:y1_end]

# Define the larger window in image 2
x2_center = x1_center
y2_center = y1_center

# Extract the larger window in image 2
x2_start = x2_center - n2 // 2
x2_end = x2_start + n2
y2_start = y2_center - n2 // 2
y2_end = y2_start + n2
patch2 = image2[x2_start:x2_end, y2_start:y2_end]

Next, we make a plot of the two SAR images, including the search and reference windows

In [ ]:
# Plot the larger image 1 showing the entire scene
plt.figure(figsize=(20, 20))
plt.imshow(image1[1500:3500,1500:3500], cmap='gray',vmin=0,vmax=500)
plt.title('(Image 1 - 26-02-2020)')
plt.axis('on')
# Add a box indicating the boundaries of the small window
plt.gca().add_patch(plt.Rectangle((y1_start, x1_start), n1, n1, linewidth=2, edgecolor='r', facecolor='none'))

# Plot the larger image 2 showing the entire scene
plt.figure(figsize=(20, 20))
plt.imshow(image2, cmap='gray',vmin=0,vmax=500)
plt.title('(Image 2 - 04-11-2020)')
plt.axis('on')
# Add a box indicating the boundaries of the larger window
plt.gca().add_patch(plt.Rectangle((y2_start, x2_start), n2, n2, linewidth=2, edgecolor='g', facecolor='none'))


Now we compute the cross correlation of the reference window with all windows of a similar size in the search window.

In [ ]:
# Compute cross-correlation of the small window with all possible subsets of size n1xn1 in the larger window
cross_correlations = np.zeros((n2 - n1 + 1, n2 - n1 + 1))
for i in range(n2 - n1 + 1):
    for j in range(n2 - n1 + 1):
        subset = patch2[i:i+n1, j:j+n1]
        cross_correlations[n2 - n1  - i, j] = compute_normalized_cross_correlation(patch1, subset)

# Find the maximum cross-correlation value and its location
max_cross_correlation = np.max(cross_correlations)
max_location = np.unravel_index(np.argmax(cross_correlations), cross_correlations.shape)

Here, we plot the resulting cross correlation values

In [ ]:
# Plot the cross-correlation values with a marker at the maximum value
plt.imshow(cross_correlations, cmap='hot', origin='lower')
plt.colorbar()
plt.scatter(max_location[1], max_location[0], color='blue', marker='x', s=100, label='Maximum')
plt.legend()
plt.title('Cross-correlation')
plt.xlabel('X')
plt.ylabel('Y')
plt.show()

print("Maximum Cross-Correlation:", max_cross_correlation)
print("Maximum Cross-Correlation Location (x, y):", max_location)

By computing the diffence between the center of the reference window and the location of maximum cross-correlation, we get the displacement in terms of pixels. By multiplying by the pixel size (10 m, in our case), we get the displacement in meters.

In [ ]:
SAR_resolution=10
np.sqrt((max_location[0]-(n2/2-n1/2))**2+(max_location[1]-(n2/2-n1/2))**2)*SAR_resolution

In [ ]:
dx = (max_location[0]-(n2/2-n1/2))*SAR_resolution
dy = (max_location[1]-(n2/2-n1/2))*SAR_resolution
print(dx,dy)

In [ ]:
# Plot the smaller image zoomed in on the location of the small window
plt.figure(figsize=(20, 20))
plt.imshow(image1, cmap='gray',vmin=0,vmax=500)
plt.title('Window Image 1 - 26-02-2020')
plt.axis('off')

# Add a box indicating the boundaries of the larger window
plt.gca().add_patch(plt.Rectangle((y2_start, x2_start), n2, n2, linewidth=2, edgecolor='g', facecolor='none'))
# Add a box indicating the boundaries of the small window
plt.gca().add_patch(plt.Rectangle((y1_start, x1_start), n1, n1, linewidth=2, edgecolor='r', facecolor='none'))

# Adjust the plot limits to focus on the larger window
plt.xlim(y2_start-100, y2_end+100)
plt.ylim(x2_end+100, x2_start-100)

# Plot the smaller image zoomed in on the location of the small window
plt.figure(figsize=(20, 20))
plt.imshow(image2, cmap='gray',vmin=0,vmax=500)
plt.title('Window Image 2 - 04-11-2020')
plt.axis('off')

# Add a box indicating the boundaries of the larger window
plt.gca().add_patch(plt.Rectangle((y2_start, x2_start), n2, n2, linewidth=2, edgecolor='g', facecolor='none'))
# Add a box indicating the boundaries of the small window
plt.gca().add_patch(plt.Rectangle((y1_start, x1_start), n1, n1, linewidth=2, edgecolor='r', facecolor='none'))


# Adjust the plot limits to focus on the larger window
plt.xlim(y2_start-100, y2_end+100)
plt.ylim(x2_end+100, x2_start-100)


# Display the plots
plt.show()